## Sentence classifier, model deployment 
Load data from model dump folder, train and store for further use in production
### Load csv

In [8]:
import sys, os, re, unidecode, random, gensim, logging
import pandas as pd
from pprint import pprint
sys.path.append('/home/od13/addons/tender_cat/libcat/mytextpipe/')
from mytextpipe import corpus
from uk_stemmer import UkStemmer
from string import punctuation 
from tqdm import tqdm
from IPython.display import clear_output

_logger = logging.getLogger(__name__)

path = '/home/od13/addons/tender_cat/data/model/dump/3'

csvs = []
for root, dirs, files in os.walk(path):
    for f in files: 
        if f.endswith(".csv"):
            try: 
                csv = pd.read_csv(os.path.join(path, f))
                csvs.append(csv)
            except (FileExistsError, IOError, pd.errors.EmptyDataError) as e:
                _logger.error('{}: {}'.format(f,e))

txt_data = pd.concat(csvs,ignore_index=True)
txt_data.fillna(value = {'label_id': 0, 'label_name': '?'}, inplace=True)
#todo: check columns, missing and duplicated

print('Loaded {} sentences from {} files'.format(txt_data.shape[0], len(set(txt_data['file'].dropna()))))



Loaded 10037 sentences from 36 files


## Normalize text

In [9]:
corp = corpus.HTMLCorpusReader('.', stemmer=UkStemmer(), clean_text=True, language='russian')


def normalize(text):
    
    text = text.strip()
    
    url_pattern = r'https?://\S+|http?://\S+|www\.\S+'
    text = re.sub(pattern=url_pattern, repl=' ', string=text)
    
    #text = unidecode.unidecode(text)
    text = text.translate(str.maketrans('', '', punctuation))
    
    number_pattern = r'\d+'
    #text = re.sub(pattern=number_pattern, repl="nmbr", string=text)
    #text = re.sub(pattern=number_pattern, repl=" ", string=text)
    
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    text = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    
    text = gensim.utils.decode_htmlentities(gensim.utils.deaccent(text))
    
    space_pattern = r'\s+'
    text = re.sub(pattern=space_pattern, repl=" ", string=text)
    
    return [x for x in corp.text_to_words(text)]

def read_corpus_df(source_df, tokens_only=False, txt_column='text'):
    for index, row in source_df.iterrows():
        word_lst = normalize(str(row[txt_column]))
        if word_lst:
            if tokens_only:
                yield word_lst
            else:
                yield gensim.models.doc2vec.TaggedDocument(word_lst, [row['doc2vec_uid']])

def ivec(word_list, model, epochs=850, alpha=0.025):
    return model.infer_vector(word_list, epochs=epochs)

def map_label(labels, name=None, index=None):
    if name is not None:
        return labels[name]
    elif index is not None:
        return list(labels.keys())[list(labels.values()).index(index)]
    else:
        raise ValueError("Specify name or number, not both")

label_dct = {}
for i, val in enumerate(list(set(txt_data['label_name'].dropna()))):
    label_dct.update({val: i})
       
txt_data['doc2vec_uid'] = range(1, len(txt_data.index)+1)
train_corpus = list(read_corpus_df(txt_data))

print('Prepeared train corpus for doc2vec, items count {}'.format(len(train_corpus)))
#pprint(train_corpus[:50])

Prepeared train corpus for doc2vec, items count 10033


## Instantinate doc2vec model

In [6]:
val_results = [7.4, 50, 100, 2] #[6.1, 250, 100, 2] [7.7, 100, 100, 2]
val_results = [6.1, 250, 100, 2]
val_results = [7.7, 100, 100, 2]
val_results = [6.1, 300, 200, 2]

print('Creating model with hyperparameters: vector_sizes {} epochs {} window {} ...'.format(val_results[1], val_results[2], val_results[3]))
%time model = gensim.models.doc2vec.Doc2Vec(train_corpus, vector_size=val_results[1], window=val_results[3], min_count=1, workers=8, epochs=val_results[2])
print('\nDoc2vec model is ready, vocabulary {}'.format(len(model.wv.vocab)))
#pprint(model.wv.vocab)

Creating model with hyperparameters: vector_sizes 300 epochs 200 window 2 ...
CPU times: user 6min 4s, sys: 1min 17s, total: 7min 21s
Wall time: 4min 21s

Doc2vec model is ready, vocabulary 7130


## Test doc2vec model
Rank by similarities, cross validate with other parameters

In [4]:
test_sent = 'У разі якщо учасник, відповідно до норм чинного законодавства не є платником податку на додану вартість або єдиного податку, такий учасник подає довідку в довільній формі із зазначенням системи оподаткування, яку він обрав, подає підтверджуючі документи (у разі наявності) та зазначає інформацію про законодавчі підстави для їх ведення.'
test_sent = 'Довідка у довільній формі про досвід виконання аналогічного (них) договору (ів) за 2020 рік, що відповідають предмету закупівлі (в разі укладання подібних договорів).'
#test_sent = 'В разі отримання мотивованої відмови Замовника від підписання Акту виконаних робіт (наданих послуг) Сторонами складається протокол, в якому вказуються зауваження і терміни їх усунення, обов\'язкові для Виконавця.'
#test_sent = 'Довідка у вигляді електронного документу із ЕЦП КЕП особи, яка уповноважена на підписання такої довідки або сканкопія папурової  довідки або сканкопія нотаріально завіреної довідки про те, що службова (посадова) особа переможця процедури закупівлі, яка підписала тендерну пропозицію, не знятої чи не погашеної судимості  не має.'
#test_sent = 'Погоджений учасником проект договору згідно Додатку 3 Оголошення.'
#test_sent = 'Інформація в довільній формі за власноручним підписом фізичної особи, яка є учасником та завірена печаткою (у разі наявності) про те, що фізична особа, яка є учасником, не була засуджена за злочин, учинений з корисливих мотивів (зокрема, повязаний з хабарництвом та відмиванням коштів), судимість з якої не знято або не погашено у встановленому законом порядку'
#test_sent = 'Копія свідоцтва про реєстрацію платника ПДВ або витягу з реєстру платників ПДВ або копія свідоцтва сплати єдиного податку або копія витягу з реєстру платників єдиного податку'

inferred_vector = ivec(normalize(test_sent), model)
sims = model.docvecs.most_similar([inferred_vector], topn=10)

found_rows = txt_data['doc2vec_uid'].isin([doc_id for doc_id, sim in sims])
#for index, row in txt_data[found_rows].iterrows():
#    print(sim, row['text'])


## Classifier
Prepare datasets

In [5]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split

txt_data['cls_uid'] = [-1 for i in range(1, len(txt_data.index)+1)]

X,y,cnt = [],[],0
for index, row in txt_data.iterrows():
    vec_uid = row['doc2vec_uid']
    if vec_uid in model.docvecs:
        text_vec = model.docvecs[vec_uid]
        X.append(text_vec)
        y.append(map_label(label_dct, name=row['label_name']))
        row['cls_uid'] = cnt
        cnt += 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

print('Full dataset {}, train {}, test {} sentences'.format(len(y), len(y_train), len(y_test)))

from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

print('Start classifier training...')

scaler = StandardScaler()
#scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
# activation = {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
# solver = {‘lbfgs’, ‘sgd’, ‘adam’}
# max_iter = 200
# learning_rate = {‘constant’, ‘invscaling’, ‘adaptive’}
clf = MLPClassifier(solver='adam', alpha=1e-7, hidden_layer_sizes=(200, 100), random_state=1, activation='logistic', max_iter = 2000,
                    learning_rate = 'adaptive', verbose=False, tol=1e-6)

# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
#clf = SVC(C=2.0, kernel='poly', degree=5, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.0001, 
#          cache_size=800, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=1)

%time y_pred = clf.fit(X_train, y_train).predict(X_test)

doc_cnt = sum([1 for lab in y_test if lab != 0])
print('Labels in tests {}'.format(doc_cnt))

print('Accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, zero_division=0,
                            target_names = list([map_label(label_dct, index=i) for i in range(0, len(label_dct))]),
                            labels=list([map_label(label_dct, name=map_label(label_dct, index=i)) for i in range(0, len(label_dct))])
     ))


Full dataset 1957, train 1761, test 196 sentences
Start classifier training...
CPU times: user 3min 4s, sys: 3min 34s, total: 6min 38s
Wall time: 50.2 s
Labels in tests 196
Accuracy 0.8928571428571429
                                                precision    recall  f1-score   support

              Ліцензії або дозвільні документи       0.00      0.00      0.00         0
          Наявність матеріально-технічної бази       1.00      1.00      1.00         2
                           Підтвердження особи       1.00      1.00      1.00         1
                           Фінансова звітність       1.00      1.00      1.00         1
                                Персональн дан       0.00      0.00      0.00         0
                            Кошторис imd(файл)       0.00      0.00      0.00         0
                               ПРОЕКТ ДОГОВІРУ       0.50      1.00      0.67         1
                           Установчі документи       0.00      0.00      0.00         2
      

## Save models to file

In [34]:
import joblib 

trained_folder = '/home/od13/addons/tender_cat/data/model/trained'
if not os.path.exists(trained_folder):
    raise FileNotFoundError('Trained model folder {} not found'.format(trained_folder))

vec_path = os.path.join(trained_folder, 'vectorizer.mod')
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
joblib.dump(model, vec_path)
print('doc2vec model saved to {}'.format(vec_path))

clf_path = os.path.join(trained_folder, 'classifier.mod')
joblib.dump(clf, clf_path) 
print('classifier saved to {}'.format(clf_path))



doc2vec model saved to /home/od13/addons/tender_cat/data/model/trained/vectorizer.mod
classifier saved to /home/od13/addons/tender_cat/data/model/trained/classifier.mod


In [35]:
#test_sent = 'Довідка у довільній формі про досвід виконання аналогічного (них) договору (ів) за 2020 рік, що відповідають предмету закупівлі (в разі укладання подібних договорів).'
#test_sent = 'В разі отримання мотивованої відмови Замовника від підписання Акту виконаних робіт (наданих послуг) Сторонами складається протокол, в якому вказуються зауваження і терміни їх усунення, обов\'язкові для Виконавця.'
#test_sent = 'Довідка у вигляді електронного документу із ЕЦП КЕП особи, яка уповноважена на підписання такої довідки або сканкопія папурової  довідки або сканкопія нотаріально завіреної довідки про те, що службова (посадова) особа переможця процедури закупівлі, яка підписала тендерну пропозицію, не знятої чи не погашеної судимості  не має.'
#test_sent = 'Погоджений учасником проект договору згідно Додатку 3 Оголошення.'
#test_sent = 'Інформація в довільній формі за власноручним підписом фізичної особи, яка є учасником та завірена печаткою (у разі наявності) про те, що фізична особа, яка є учасником, не була засуджена за злочин, учинений з корисливих мотивів (зокрема, повязаний з хабарництвом та відмиванням коштів), судимість з якої не знято або не погашено у встановленому законом порядку'
#test_sent = 'Копія свідоцтва про реєстрацію платника ПДВ або витягу з реєстру платників ПДВ або копія свідоцтва сплати єдиного податку або копія витягу з реєстру платників єдиного податку'
#test_sent = 'Копія виписки з Єдиного державного реєстру юридичних осіб'
#test_sent = 'Оригінал або нотаріально завірена копія документу (-ів) видану уповноваженим органам про те, що, службова (посадова) особа учасника, яку уповноважено учасником представляти його інтереси під час проведення процедури закупівлі не була засуджена за злочин, вчинений з корисливих мотивів, судимість з якої не знято або не погашено у встановленому законом порядку.'
#test_sent = 'asdvasbasdfnbsadfbn'

model = joblib.load(vec_path)
clf = joblib.load(clf_path)

empty_label = '?'
true_positive,false_positive = [],[]
cnt,label_cnt,false_cnt = 0,0,0
pbar = tqdm(range(1, txt_data.shape[0]))
for i in pbar:
    text = txt_data['text'][i]
    if not isinstance(text, str):
        continue
    
    text_vector = ivec(normalize(text), model)
    prediction = clf.predict(scaler.transform([text_vector]))
    label_name = map_label(label_dct, index=prediction[0])
    if label_name != empty_label:
        label_cnt += 1
    
    if prediction[0] != map_label(label_dct, name=empty_label):
        
        if label_name == txt_data['label_name'][i]:
            true_positive.append((map_label(label_dct, index=prediction[0]), text))
            cnt += 1
        else:
            false_positive.append((map_label(label_dct, index=prediction[0]), text, txt_data['label_name'][i]))
            false_cnt +=1
        pbar.set_postfix({'docs found {} of total {}, false items {}'.format(cnt, label_cnt, false_cnt): (cnt/label_cnt)*100})
            
        if cnt == 10:
            break
        
print('Found sentences about documents {} from total {}'.format(len(true_positive), label_cnt))

for doc in false_positive:
    print('---- (pred) {} (real) {}:\n{}'.format(doc[0], doc[2],doc[1]))


  5%|▍         | 95/1956 [00:05<01:55, 16.11it/s, docs found 10 of total 15, false items 5=66.7]

Found sentences about documents 10 from total 15
---- (pred) Кваліфікаційні довідки (real) ?:
Місцезнаходження (адреса)
---- (pred) Технічні вимоги (real) ?:
Інформація про предмет закупівлі:
---- (pred) Цінова пропозиція (real) ?:
Інформація про мову (мови), якою (якими) повинно бути складено тендерні пропозиції
---- (pred) Цінова пропозиція (real) ?:
Унесення змін до тендерної документації.
---- (pred) Довідка по статті 17 (real) Кваліфікаційні довідки:
'- інформації та документів, що підтверджують відповідніст.учасника кваліфікаційним критеріям;
